*Copyright 2019 StarkWare Industries Ltd.<br> Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.starkware.co/open-source-license/ <br> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.*

# Part 1: Trace and Low-Degree Extension

- [Video Lecture (youtube)](https://www.youtube.com/watch?v=Y0uJz9VL3Fo)
- [Slides (PDF)](https://starkware.co/wp-content/uploads/2021/12/STARK101-Part1.pdf)

Today we will develop a STARK prover for the FibonacciSq sequence over a finite field.
The FibonacciSq sequence is defined by the recurrence relation $a_{n+2} = a_{n+1} ^2 + a_n ^2$.
<br>By the end of the day, your code will produce a *STARK* proof attesting to the following statement: <br>**I know a field element $X\in \mathbb{F}$ such that the 1023rd element of the FibonacciSq sequence starting with $1, X$ is $2338775057$**.
<br><br>
## The Basics
### FieldElement class
We use our `FieldElement` class to represent field elements.<br> You can construct instances of `FieldElement` from integers, and then add, multiply, divide, get inverse, and so on.
The underlying field of this class is $\mathbb{F}_{3221225473}$ ($3221225473 = 3 \cdot 2^{30} + 1$), so all operations are done modulo 3221225473.
<br><br>
Try it by running the following cell (shift + enter):

In [1]:
from field import FieldElement
FieldElement(3221225472) + FieldElement(10)

9

In [2]:
3*(2**30)+1

3221225473

## FibonacciSq Trace

To start, let's construct a list `a` of length 1023, whose first two elements will be FieldElement objects representing 1 and 3141592, respectively. The next 1021 elements will be the FibonacciSq sequence induced by these two elements. `a` is called the trace of FibonacciSq, or, when the context is clear, the trace. <br>
Correct the code below to fill `a`:

In [5]:
a = [FieldElement(1), FieldElement(3141592)]
b = a[0] * a[0] + a[1] * a[1]
b

-234554807

In [12]:
(a[1].val * a[1].val) % FieldElement.k_modulus

2986670665

In [14]:
a[1] * a[1]

-234554808

In [16]:
print((a[1].val * a[1].val) % FieldElement.k_modulus)
print((a[1] * a[1]).val)
print(a[1] * a[1])

2986670665
2986670665
-234554808


Solution (click to the ... to unhide):

In [6]:
a = [FieldElement(1), FieldElement(3141592)]
while len(a) < 1023:
    a.append(a[-2] * a[-2] + a[-1] * a[-1])

In [7]:
a

[1,
 3141592,
 -234554807,
 -67434632,
 -9942284,
 -912015755,
 360534264,
 1521485062,
 250071759,
 -1164398925,
 -1278840892,
 -1133908886,
 -281696034,
 -1084927914,
 241328674,
 -283356885,
 -499510474,
 -1393358017,
 662115204,
 876880377,
 11010363,
 134266761,
 1013531292,
 -1589436994,
 -1535876426,
 1099128259,
 -1494718479,
 -113558994,
 -874749121,
 818474384,
 1470614342,
 1137056828,
 45628645,
 -1415376709,
 -1479888288,
 -815018590,
 -846089645,
 424463880,
 -432411915,
 948208429,
 1514483688,
 -1164827390,
 471624747,
 92605242,
 -375909078,
 -1013188985,
 551915003,
 1477216461,
 -266655678,
 652312361,
 743174813,
 -452000314,
 -515866803,
 -529632378,
 821677153,
 24606723,
 1550353242,
 -787122521,
 284758060,
 -972359745,
 575461207,
 -740688016,
 1195646405,
 541324451,
 827815664,
 325126819,
 -1581881409,
 1028925401,
 1079419294,
 -375012757,
 -610596127,
 1358984093,
 135288188,
 -1212224098,
 664398479,
 462537441,
 -923593347,
 1452891844,
 -6270003,
 -1275

### Test Your Code
Run the next cell to test that you have filled `a` correctly.<br> Note that this is in fact a verifier, albeit very naive and non-succinct one, as it goes over the sequence, element by element, making sure it is correct.

In [18]:
a[1022].val

2338775057

In [ ]:
assert len(a) == 1023, 'The trace must consist of exactly 1023 elements.'
assert a[0] == FieldElement(1), 'The first element in the trace must be the unit element.'
for i in range(2, 1023):
    assert a[i] == a[i - 1] * a[i - 1] + a[i - 2] * a[i - 2], f'The FibonacciSq recursion rule does not apply for index {i}'
assert a[1022] == FieldElement(2338775057), 'Wrong last element!'
print('Success!')

## Thinking of Polynomials
We now want to think of the sequence as the evaluation of some, yet unknown, polynomial $f$ of degree 1022 (due to the Unisolvence Theorem).
We will choose the domain to be some subgroup $G \subseteq \mathbb{F}^\times$ of size 1024, for reasons that will become clear later.

(Recall that $\mathbb{F}^\times$ denotes the multiplicative group of $\mathbb{F}$, which we get from $\mathbb{F}$ by omitting the zero element with the induced multiplication from the field. A subgroup of size 1024 exists because $\mathbb{F}^\times$ is a cyclic group of size $3\cdot 2^{30}$, so it contains a subgroup of size $2^i$ for any $0 \leq i \leq 30$).
### Find a Group of Size 1024
If we find an element $g \in \mathbb{F}$ whose (multiplicative) order is 1024, then $g$ will generate such a group.
The class `FieldElement` provides a static method `generator()` which returns an element that generates $\mathbb{F}^\times$ (whose order is $|\mathbb{F}^\times|$).
1. Use it to obtain a generator $g$ for $G$.
2. Create a list called `G` with all the elements of $G$, such that $G[i] := g^i$.

*Hint: When $k$ divides $|\mathbb{F}^\times|$, $g^k$ generates a group of size $\frac {|\mathbb{F}^\times|}{k}$, and the n-th power of some `FieldElement` $x$ can be computed by calling `x ** n `.*



In [ ]:
# Change the following line so that g will generate a group of size 1024
g = FieldElement.generator()
# Fill G with the elements of G such that G[i] := g ** i
G = [] 

Solution:

In [ ]:
g = FieldElement.generator() ** (3 * 2 ** 20)
G = [g ** i for i in range(1024)]

Run the next cell to test your code. 

In [ ]:
# Checks that g and G are correct.
assert g.is_order(1024), 'The generator g is of wrong order.'
b = FieldElement(1)
for i in range(1023):
    assert b == G[i], 'The i-th place in G is not equal to the i-th power of g.'
    b = b * g
    assert b != FieldElement(1), f'g is of order {i + 1}'
    
if b * g == FieldElement(1):
    print('Success!')
else:
    print('g is of order > 1024')

### Polynomial class
We provide you with a class called `Polynomial`. The simplest way to construct a `Polynomial` is by using the variable `X` (note that it's a capital `X`) which represents the formal variable $x$:

In [ ]:
from polynomial import X
# The polynomial 2x^2 + 1.
p = 2*X**2 + 1
# Evaluate p at 2:
print(p(2))
# Type a polynomial's name, on its own, in the last line of a cell to display it
p

### Interpolating a Polynomial
Our `polynomial` module provides a Lagrange interpolation function, whose arguments are:
* x_values: x-values of G that the polynomial's values for them is known. [List]
* y_values: the corresponding y-values. [List]

It returns the unique `Polynomial` of degree < `len(x_values)` instance that evaluates to `y_values[i]` on `x_values[i]` for all i.

Run the following cell to get help on the function `interpolate_poly`.

In [ ]:
from polynomial import interpolate_poly
interpolate_poly?

Suppose that `a` contains the values of some polynomial over `G` (except for `G[-1]`, since `a` is one element shorter).
Use `interpolate_poly()` to get `f` and get its value at `FieldElement(2)`. 

In [ ]:
from polynomial import interpolate_poly
# Fix the following so that v will contain the value of f at FieldElement(2)
# Note that interpolate_poly may take up to a minute to run.
v = 'YOUR_CODE_HERE'

Solution:

In [ ]:
f = interpolate_poly(G[:-1], a)
v = f(2)

Run test:

In [ ]:
assert v == FieldElement(1302089273)
print('Success!')

## Evaluating on a Larger Domain
The trace, viewed as evaluations of a polynomial $f$ on $G$, can now be extended by evaluating $f$ over a larger domain, thereby creating a Reed-Solomon error correction code.

### Cosets
To that end, we must decide on a larger domain on which $f$ will be evaluated. 
We will work with a domain that is 8 times larger than $G$. <br>A natural choice for such a domain is to take some group $H$ of size 8192 (which exists because 8192 divides $|\mathbb{F}^\times|$), and shift it by the generator of $\mathbb{F}^\times$, thereby obtaining a [coset](https://en.wikipedia.org/wiki/Coset) of $H$.

Create a list called `H` of the elements of $H$, and multiply each of them by the generator of $\mathbb{F}^\times$ to obtain a list called `eval_domain`. In other words, eval_domain = $\{w\cdot h^i | 0 \leq i <8192  \}$ for $h$ the generator of $H$ and $w$ the generator of $\mathbb{F}^\times$.

Hint: You already know how to obtain $H$ - similarly to the way we got $G$ a few minutes ago.


In [ ]:
# Fix the following, make sure that the the element of h are powers of its generator in 
# order, that is - H[0] will be the unit, H[1] will be h (H's generator), H[2] will be H's
# generator squared, etc.
H = 'YOUR_CODE_HERE'
eval_domain = 'YOUR_CODE_HERE'

Solution:

In [ ]:
w = FieldElement.generator()
h = w ** ((2 ** 30 * 3) // 8192)
H = [h ** i for i in range(8192)]
eval_domain = [w * x for x in H]


Run test:

In [ ]:
from hashlib import sha256
assert len(set(eval_domain)) == len(eval_domain)
w = FieldElement.generator()
w_inv = w.inverse()
assert '55fe9505f35b6d77660537f6541d441ec1bd919d03901210384c6aa1da2682ce' == sha256(str(H[1]).encode()).hexdigest(),\
    'H list is incorrect. H[1] should be h (i.e., the generator of H).'
for i in range(8192):
    assert ((w_inv * eval_domain[1]) ** i) * w == eval_domain[i]
print('Success!')

### Evaluate on a Coset
Time to use `interpolate_poly` and `Polynomial.poly` to evaluate over the coset. Note that it is implemented fairely naively in our Python module, so interpolation may take up to a minute.<br>
Indeed - interpolating and evaluating the trace polynomial is one of the most computationally-intensive steps in the STARK protocol, even when using more efficient methods (e.g. FFT).

In [ ]:
# Fill f_eval with the evaluations of f on eval_domain.
f_eval = 'YOUR_CODE_HERE'

Solution:

In [ ]:
f = interpolate_poly(G[:-1], a)
f_eval = [f(d) for d in eval_domain]

Run test:

In [ ]:
# Test against a precomputed hash.
from hashlib import sha256
from channel import serialize
assert '1d357f674c27194715d1440f6a166e30855550cb8cb8efeb72827f6a1bf9b5bb' == sha256(serialize(f_eval).encode()).hexdigest()
print('Success!')

## Commitments
We will use [Sha256](https://en.wikipedia.org/wiki/SHA-2)-based [Merkle Trees](https://en.wikipedia.org/wiki/Merkle_tree) as our commitment scheme.
A simple implementation of it is available to you in the `MerkleTree` class.
Run the next cell (for the sake of this tutorial, this also serves as a test for correctness of the entire computation so far):

In [ ]:
from merkle import MerkleTree
f_merkle = MerkleTree(f_eval)
assert f_merkle.root == '6c266a104eeaceae93c14ad799ce595ec8c2764359d7ad1b4b7c57a4da52be04'
print('Success!')

## Channel
Theoretically, a STARK proof system is a protocol for interaction between two parties - a prover and a verifier. In practice, we convert this interactive protocol into a non-interactive proof using the [Fiat-Shamir Heuristic](https://en.wikipedia.org/wiki/Fiat%E2%80%93Shamir_heuristic). In this tutorial you will use the `Channel` class, which implements this transformation. This channel replaces the verifier in the sense that the prover (which you are writing) will send data, and receive random numbers or random `FieldElement` instances.

This simple piece of code instantiates a channel object, sends the root of your Merkle Tree to it. 
Later, the channel object can be called to provide random numbers or random field elements.

In [ ]:
from channel import Channel
channel = Channel()
channel.send(f_merkle.root)

Lastly - you can retrieve the proof-so-far (i.e., everything that was passed in the channel up until a certain point) by printing the member `Channel.proof`.

In [ ]:
print(channel.proof)